## 코드 추상화: 클래스와 객체 2부

안내: [python-textbook.readthedocs.io의 Classes](https://python-textbok.readthedocs.io/en/1.0/Classes.html#class-attributes) 내용을 요약 및 수정한 내용입니다.

## 클래스 장식자

**장식자**(decorator)는 다른 함수의 기능에 다른 기능을 추가할 때 사용되는 함수이다.
즉, 장식자는 함수를 인자로 받아 그 함수가 하는 일에 더해 다른 기능도
수행하는 함수를 리턴값으로 내준다.
이런 장식자를 함수로 정의할 수 있는 이유는 함수가 제1종 객체이기 때문이다.
즉, 다른 함수의 인자 또는 리턴값으로 사용될 수 있다.

파이썬에서 제공하는 장식자가 매우 다양하며, 사용자가 직접 장식자를 정의할 수도 있다.
여기서는 클래스에서 선언된 메서드의 종류를 구분하기 위해 사용되는 
두 개의 장식자
`@classmethod`와 `@staticmethod`를 소개한다. 

### 메서드 종류

클래스에서 선언된 메서드는 세 종류로 나뉜다.

1. 인스턴스 메서드(instance method)
    * 클래스의 인스턴스가 생성된 후에 객체와 함께 사용되는 메서드
    * 첫째 매개변수로 `self` 사용 (관습)
1. 클래스 메서드(class method)
    * 인스턴스 없이 클래스 이름과 함께 사용될 수 있는 메서드
    * 첫째 매개변수로 `cls` 사용 (관습)
    * 클래스 내의 모든 속성과 메서드를 `cls` 지정자와 함께 사용 가능
    * 단, 인스턴스 속성 사용 불가.
    * 해당 클래스의 모든 인스턴스에서도 사용 가능
1. 정적 메서드(static method)
    * 인스턴스 없이 클래스 이름과 함께 사용될 수 있는 메서드
    * 첫째 매개변수에 대한 의무사항 없음.
    * 클래스 내의 모든 속성과 메서드를 클래스 이름을 지정자로 사용하여 사용해야 함.
    * 단, 인스턴스 속성 사용 불가.
    * 해당 클래스의 모든 인스턴스에서도 사용 가능
    
인스턴스 메서드는 특별한 장식자가 필요 없다.
예를 들어, 지금까지 살펴본 모든 메서드는 인스턴스 메서드이다. 

### 클래스 메서드 장식자: `@classmethod`

클래스의 모든 인스턴스간에 공유되는 클래스 속성을 정의 할 수있는 것처럼 클래스 메소드를 정의 할 수 있습니다 . 우리는 @classmethod데코레이터를 사용하여 일반적인 방법을 장식 함으로써이 작업을 수행합니다 .

클래스 메소드에는 여전히 호출 객체가 첫 번째 매개 변수로 있지만 규칙에 따라이 매개 변수의 이름을에서 self로 변경 합니다 cls. 인스턴스에서 클래스 메서드를 호출하면이 매개 변수에 인스턴스 객체가 포함되지만 클래스에서 호출하면 클래스 객체가 포함됩니다. 매개 변수를 호출함으로써 인스턴스 속성 cls이 보장되지는 않는다는 것을 상기시킵니다 .

좋은 수업 방법은 무엇입니까? 때로는 클래스 인스턴스를 만들 필요없이 상수 및 기타 클래스 특성을 사용하여 수행 할 수있는 클래스와 관련된 작업이 있습니다. 이러한 작업에 인스턴스 메서드를 사용해야한다면 아무 이유없이 인스턴스를 만들어야하므로 낭비가됩니다. 때때로 우리는 관련 상수를 그에 작용하는 함수와 함께 그룹화하기 위해 순수하게 클래스를 작성합니다. 이러한 클래스를 전혀 인스턴스화 할 수는 없습니다.

입력을 처리 한 후 클래스의 인스턴스를 작성하는 클래스 메소드를 작성하여 클래스 생성자에 전달할 올바른 형식으로 작성하는 것이 유용한 경우도 있습니다. 이를 통해 생성자는 간단하고 복잡한 구문 분석 또는 정리 코드를 구현할 필요가 없습니다.

In [1]:
class Person:

    def __init__(self, name, surname, birthdate, address, telephone, email):
        self.name = name
        # (...)

    @classmethod
    def from_text_file(cls, filename):
        # extract all the parameters from the text file
        return cls(*params) # this is the same as calling Person(*params)

### 정적 메서드 장식자: `@staticmethod`

정적 메서드에는 호출 객체가 첫 번째 매개 변수로 전달되지 않습니다. 즉, 나머지 클래스 나 인스턴스에 전혀 액세스 할 수 없습니다. 인스턴스 또는 클래스 객체에서 호출 할 수 있지만 클래스 메서드와 같은 클래스 객체에서 가장 일반적으로 호출됩니다.

클래스를 사용하여 서로 또는 클래스의 다른 데이터에 액세스 할 필요가없는 관련 메소드를 그룹화하는 경우이 기술을 사용할 수 있습니다. 정적 메소드를 사용할 때의 장점은 불필요한 제거이다 cls또는 self우리의 방법에서 정의 파라미터. 단점은 정적 메소드 내에서 다른 클래스 메소드 또는 속성을 가끔 참조하려는 경우 클래스 이름을 완전히 작성해야하며, 이는 cls클래스 내부에서 사용 가능한 변수를 사용하는 것보다 훨씬 더 장황 할 수 있다는 것입니다 방법.

다음은 세 가지 방법 유형을 비교하는 간단한 예입니다.

In [2]:
class Person:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self): # instance method
        # instance object accessible through self
        return "%s %s" % (self.name, self.surname)

    @classmethod
    def allowed_titles_starting_with(cls, startswith): # class method
        # class or instance object accessible through cls
        return [t for t in cls.TITLES if t.startswith(startswith)]

    @staticmethod
    def allowed_titles_ending_with(endswith): # static method
        # no parameter for class or instance object
        # we have to use Person directly
        return [t for t in Person.TITLES if t.endswith(endswith)]


jane = Person("Jane", "Smith")

print(jane.fullname())

print(jane.allowed_titles_starting_with("M"))
print(Person.allowed_titles_starting_with("M"))

print(jane.allowed_titles_ending_with("s"))
print(Person.allowed_titles_ending_with("s"))

Jane Smith
['Mr', 'Mrs', 'Ms']
['Mr', 'Mrs', 'Ms']
['Mrs', 'Ms']
['Mrs', 'Ms']


### `@property`

때로는 메소드를 사용하여 객체의 속성을 동적으로 생성하여 객체의 다른 속성에서 계산합니다. 때로는 메소드를 사용하여 단일 속성에 액세스하여 리턴 할 수도 있습니다. 다른 방법을 사용하여 속성에 직접 액세스하는 대신 속성 값을 업데이트 할 수도 있습니다. 이와 같은 메소드 는 속성 값을 각각 "get"및 "set"하므로 getter 및 setter 라고합니다.

일부 언어에서는 모든 속성에 대해 getter 및 setter를 사용하고 해당 값에 직접 액세스하지 않는 것이 좋습니다. 그리고 setter 및 getter를 통한 경우를 제외하고는 속성에 액세스 할 수없는 언어 기능이 있습니다. 파이썬에서 간단한 속성에 직접 액세스하는 것은 완벽하게 허용되며, 모든 속성에 대해 게터와 세터를 작성하는 것은 불필요하게 장황한 것으로 간주됩니다. 세터는 복합 할당 연산자를 사용할 수 없기 때문에 불편할 수 있습니다.

In [6]:
class Person:
    def __init__(self, height):
        self.height = height

    def get_height(self):
        return self.height

    def set_height(self, height):
        self.height = height

jane = Person(153) # Jane is 153cm tall

jane.height += 1 # Jane grows by a centimetre
jane.set_height(jane.height + 1) # Jane grows again

보시다시피, setter를 통해 height 속성을 늘리는 것이 훨씬 더 장황합니다. 물론 우리 는 주어진 매개 변수에 의해 속성을 증가 시키는 두 번째 setter를 작성할 수 있습니다. 그러나 모든 속성과 수행하려는 모든 종류의 수정에 대해 비슷한 작업을 수행해야합니다. 목록에 값을 추가하는 것과 같이 내부 수정과 비슷한 문제가 있습니다.

종종 setter 및 getter 의 이점 으로 간주되는 것은 객체를 사용하는 코드에 영향을 미치지 않고 객체 내부에서 속성이 생성되는 방식을 변경할 수 있다는 것입니다. 예를 들어, 우리가 처음에 만든 가정 Person이 클래스 fullname의 속성을, 나중에 우리는 분리하도록 클래스를 변경하고자 name하고 surname우리가 이름을 만들 결합 특성을. 항상 fullnamesetter를 통해 속성에 액세스하면 setter를 다시 작성할 수 있습니다. setter를 호출하는 코드는 변경하지 않아도됩니다.

그러나 코드가 fullname속성에 직접 액세스하면 어떻게됩니까? fullname올바른 값을 반환 하는 메서드를 작성할 수 있지만 메서드를 호출해야 합니다. 다행히도 @property데코레이터를 사용하면 메소드가 속성처럼 동작하도록 할 수 있습니다.

In [4]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @property
    def fullname(self):
        return "%s %s" % (self.name, self.surname)

jane = Person("Jane", "Smith")
print(jane.fullname) # no brackets!

Jane Smith


속성에 대한 setter와 deleter를 정의하는 데 사용할 수있는 데코레이터도 있습니다 (삭제자는 객체에서 속성을 삭제합니다). getter, setter 및 deleter 메소드는 모두 동일한 이름을 가져야합니다.

In [12]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @property
    def fullname(self):
        return "%s %s" % (self.name, self.surname)

    @fullname.setter
    def fullname(self, value):
        # this is much more complicated in real life
        name, surname = value.split(" ", 1)
        self.name = name
        self.surname = surname

    @fullname.deleter
    def fullname(self):
        del self.name
        del self.surname

jane = Person("Jane", "Smith")
print(jane.fullname)

jane.fullname = "Jane Doe"
print(jane.fullname)
print(jane.name)
print(jane.surname)

del jane.fullname

Jane Smith
Jane Doe
Jane
Doe


In [13]:
jane.name

AttributeError: 'Person' object has no attribute 'name'

### 연습 4

1. 라는 클래스 만들기 Numbers라는 하나의 클래스 속성이, MULTIPLIER그리고 매개 변수를 사용하는 생성자 x와 y(이 모든 숫자이어야한다).
    1. add속성 x과 의 합을 반환하는 라는 메서드를 작성하십시오 y.
    1. 라는 클래스 메소드 쓰기 multiply하나의 숫자 매개 변수를 사용, a및 제품을 반환 a과 MULTIPLIER.
    1. subtract두 개의 숫자 매개 변수를 사용 b하고 -를 c리턴 하는 정적 메소드를 작성하십시오 .bc
    1. and value의 값을 포함하는 튜플을 반환하는 라는 메서드를 작성하십시오 . 속성에이 방법을 확인하고 세터와의 값을 조작하기위한 Deleter가 쓰기 와 .xyxy

## 매직 메서드

## 매직 메서드 재정의